# Fuzzy Melody Generator (ver 1.0)

***

### Credit for FuzzyWuzzy string matching package goes out @seatgeek: https://github.com/seatgeek/fuzzywuzzy

### Powered by tegridy-tools TMIDI 2.0 Optimus Processors

***

#### Project Los Angeles
#### Tegridy Code 2021

***

# Setup environment

In [ ]:
#@title Install dependencies
!git clone https://github.com/asigalov61/tegridy-tools
!pip install unidecode
!pip install tqdm

!pip install fuzzywuzzy[speedup]

In [ ]:
#@title Load needed modules
print('Loading needed modules. Please wait...')

import sys
import os
import json
import secrets

os.chdir('/content/tegridy-tools/tegridy-tools/')
import TMIDI
os.chdir('/content/')

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import tqdm.auto
from tqdm import auto
import copy

from google.colab import output, drive

print('Creating Dataset dir...')
if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

os.chdir('/content/')
print('Loading complete. Enjoy! :)')

# Download a sample dataset

In [ ]:
#@title Download Melody ABC Tunes Dataset
%cd /content/Dataset

# Melody-ABC-Tunes-Dataset-CC-BY-NC-SA
!wget --no-check-certificate -O Melody-ABC-Tunes-Dataset-CC-BY-NC-SA.zip "https://onedrive.live.com/download?cid=8A0D502FC99C608F&resid=8A0D502FC99C608F%2118479&authkey=AI87E6Wlb6Qfp3w"
!unzip -j Melody-ABC-Tunes-Dataset-CC-BY-NC-SA.zip
!rm Melody-ABC-Tunes-Dataset-CC-BY-NC-SA.zip

%cd /content/

# Process the MIDI dataset

## NOTE: If you are not sure what settings to select, please use original defaults

In [ ]:
#@title Process MIDIs to special MIDI dataset with Tegridy MIDI Processor
#@markdown NOTES:

#@markdown 1) Dataset MIDI file names are used as song names. Feel free to change it to anything you like.

#@markdown 2) Best results are achieved with the single-track, single-channel, single-instrument MIDI 0 files with plain English names (avoid special or sys/foreign chars)

#@markdown 3) MIDI Channel = -1 means all MIDI channels. MIDI Channel = 16 means all channels will be processed. Otherwise, only single indicated MIDI channel will be processed.

file_name_to_output_dataset_to = "/content/Fuzzy-Melody-Generator-Dataset" #@param {type:"string"}
desired_MIDI_channel_to_process = 0 #@param {type:"slider", min:-1, max:16, step:1}
encode_MIDI_channels = False #@param {type:"boolean"}
encode_velocities = False #@param {type:"boolean"}
chordify_input_MIDIs = False #@param {type:"boolean"}
time_denominator = 1 #@param {type:"slider", min:1, max:20, step:1}
chars_encoding_offset = 30000 #@param {type:"number"}

print('TMIDI Processor')
print('Starting up...')

###########

average_note_pitch = 0
min_note = 127
max_note = 0

files_count = 0

ev = 0

chords_list_f = []
melody_list_f = []

chords_list = []
chords_count = 0

melody_chords = []
melody_count = 0

TXT_String = 'DATASET=Optimus-Virtuoso-Music-Dataset' + chr(10)

TXT = ''
melody = []
chords = []
bf = 0
###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset/"
os.chdir(dataset_addr)
filez = os.listdir(dataset_addr)

print('Processing MIDI files. Please wait...')
for f in tqdm.auto.tqdm(filez):
  try:
    fn = os.path.basename(f)
    fn1 = fn.split('.')[0]
    #notes = song_notes_list[song_notes_list.index(fn1)+1]


    files_count += 1
    TXT, melody, chords = TMIDI.Optimus_MIDI_TXT_Processor(f, 
                                                           line_by_line_output=False, 
                                                           chordify_TXT=chordify_input_MIDIs, 
                                                           output_MIDI_channels=encode_MIDI_channels, 
                                                           char_offset=chars_encoding_offset, 
                                                           dataset_MIDI_events_time_denominator=time_denominator, 
                                                           output_velocity=encode_velocities, 
                                                           MIDI_channel=desired_MIDI_channel_to_process,
                                                           MIDI_patch=range(0,127))
    melody_list_f += melody
    chords_list_f += chords
    
    #TXT_String += 'INTRO=' + notes + '\n'
    TXT_String += TXT
  
  except KeyboardInterrupt:
    print('Exiting...Saving progress...')
    break

  except:
    bf += 1
    print('Bad MIDI:', f)
    print('Count:', bf)
    
    continue

print('Task complete :)')
print('==================================================')
print('Number of processed dataset MIDI files:', files_count)
print('Number of MIDI chords recorded:', len(chords_list_f))
print('First chord event:', chords_list_f[0], 'Last chord event:', chords_list_f[-1]) 
print('Number of recorded melody events:', len(melody_list_f))
print('First melody event:', melody_list_f[0], 'Last Melody event:', melody_list_f[-1])
print('Total number of MIDI events recorded:', len(chords_list_f) + len(melody_list_f))

# Writing dataset to TXT file
with open(file_name_to_output_dataset_to + '.txt', 'wb') as f:
  f.write(TXT_String.encode('utf-8', 'replace'))
  f.close

# Dataset
MusicDataset = [chords_list_f, melody_list_f]

# Writing dataset to pickle file
TMIDI.Tegridy_Pickle_File_Writer(MusicDataset, file_name_to_output_dataset_to)

# Load/Re-load

In [ ]:
#@title Load/Re-load processed dataset
full_path_to_processed_dataset = "/content/Fuzzy-Melody-Generator-Dataset" #@param {type:"string"}


# Writing dataset to memory
chords_list_f, melody_list_f = TMIDI.Tegridy_Pickle_File_Loader(full_path_to_processed_dataset)


# Prepare melodies for generation

In [ ]:
#@title Process melody list to melody slices
desired_melody_baseline_pitch = 60 #@param {type:"slider", min:1, max:127, step:1}
minimum_length_of_considered_slices = 8 #@param {type:"slider", min:4, max:32, step:4}


print('Creating pitches list...')
pitches = []

for m in auto.tqdm(melody_list_f):
  pitches.append(m[4])

output = []

pat = desired_melody_baseline_pitch # start and end pitch
min_length = minimum_length_of_considered_slices # min number of notes in a slice

out = []

print('Processing melody list to slices...')

for i in auto.tqdm(range(pitches.index(pat), len(pitches)-1)):

  if pitches[i+1] != pat:
    out.append(melody_list_f[i])
  else:
    out.append(melody_list_f[i+1])
    if len(out) >= min_length:
      out.sort(reverse=False, key=lambda x: x[1])
      output.append(out)
    out = []

print('Done! Enjoy! :)')

In [ ]:
#@title Create slices ratings list

print('Creating slices ratings list...')
print('Please wait as it may take a while...')

ratings = []

src = output[secrets.randbelow(len(output))]
for o in auto.tqdm(output):
  ratings.append(fuzz.ratio(src, o))

print('Finishing up...')
print('Max slice rating:', max(ratings))
print('Min slice rating:', min(ratings))

print('Done! Enjoy! :)') 

In [ ]:
#@title Generate melody
number_of_melody_slices = 10 #@param {type:"slider", min:5, max:100, step:5}
minimum_slice_rating_to_consider = 90 #@param {type:"slider", min:1, max:100, step:1}

print('Generating melody...')
print('Searching for right melody slices...')

number_of_slices = number_of_melody_slices
min_rating = minimum_slice_rating_to_consider

idx = 0

output_song = []
output_song.extend(src)
for i in auto.tqdm(range(number_of_slices)):
  while ratings[idx] < min_rating:

      idx = secrets.randbelow(len(output))

  sli = output[idx][:-1]
  output_song.extend(sli)
  idx = 0

print('Assembling chosen slices into final melody...')

song = []

previous_event = output_song[0]
previous_event[1] = 0
for o in auto.tqdm(output_song[1:]):
  b = copy.deepcopy(o)

  b[1] = previous_event[1] + previous_event[2]
 
  song.append(b)

  previous_event = b

print('Done! Enjoy! :)')

# Convert generated music composition to MIDI file and download/listen to the output :)

In [ ]:
#@title Convert to MIDI from TXT (w/Tegridy MIDI-TXT Processor)

#@markdown Standard MIDI timings are 400/120(80)

'''For debug:'''

#fname = '/content/Optimus-VIRTUOSO-Composition-generated-on-2021-02-25_00_45_41_715972'
#with open(fname + '.txt', 'r') as f:
#  completion = f.read()
fname = '/content/Markovify-Piano-Music-Composition'
#completion = Output_TXT_String

#completion = TXT_String[:1500]
completion = song
song_name = 'TEST'

number_of_ticks_per_quarter = 420 #@param {type:"slider", min:10, max:500, step:10}
dataset_time_denominator = 1 #@param {type:"slider", min:1, max:20, step:1}
encoding_has_MIDI_channels = False #@param {type:"boolean"}
encoding_has_velocities = False #@param {type:"boolean"}
simulate_velocity = True #@param {type:"boolean"}
chars_encoding_offset_used_for_dataset = 30000 #@param {type:"number"}

print('Converting TXT to MIDI. Please wait...')
print('Converting TXT to Song...')
'''output_list, song_name = TMIDI.Tegridy_Optimus_TXT_to_Notes_Converter(completion, 
                                                                has_MIDI_channels=encoding_has_MIDI_channels, 
                                                                simulate_velocity=simulate_velocity,
                                                                char_encoding_offset=chars_encoding_offset_used_for_dataset,
                                                                save_only_first_composition=True,
                                                                dataset_MIDI_events_time_denominator=dataset_time_denominator,
                                                                has_velocities=encoding_has_velocities,
                                                                line_by_line_dataset=False)'''

print('Converting Song to MIDI...')

output_signature = 'Markovify Piano'

detailed_stats = TMIDI.Tegridy_SONG_to_MIDI_Converter(song,
                                                      output_signature = output_signature,  
                                                      output_file_name = fname, 
                                                      track_name=song_name, 
                                                      number_of_ticks_per_quarter=number_of_ticks_per_quarter,
                                                      )

print('Done!')

print('Downloading your composition now...')
from google.colab import files
files.download(fname + '.mid')

print('Detailed MIDI stats:')
detailed_stats

# Congrats! You did it! :)